# CS 178 Project

Bisher Kudaimi 76592646

Nabhan Khan 10986609

Mustafa Yesilyurt 64496863

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

np.random.seed(0)

X = np.genfromtxt('X_train.txt', delimiter = ',')
Y = np.genfromtxt('Y_train.txt', delimiter = ',')
X,Y = ml.shuffleData(X,Y)

In [2]:
print(X.shape, Y.shape)

(7423, 107) (7423,)


In [10]:
Xtr,Xva,Ytr,Yva=ml.splitData(X,Y,0.8)

In [3]:
def toKaggle(filename,YeHat):
    fh = open(filename,'w') # open file for upload
    fh.write('ID,Predicted\n') # output header line
    for i,yi in enumerate(YeHat.ravel()):
        fh.write('{},{}\n'.format(i,yi)) # output each prediction
    fh.close()

class dummy(ml.classifier):
    def __init__(self,X,Y,P): self.P=P; self.classes=np.unique(Y);
    def predictSoft(self,X): return self.P

In [12]:
# the one to use!!!!!!!!!!!



nUse= 274
mu = Ytr.mean()
step = 0.05
Ft = np.zeros((Xtr.shape[0],)) + np.log(mu/(1-mu))
Fv = np.zeros((Xva.shape[0],)) + np.log(mu/(1-mu))
#Fe = np.zeros((Xe.shape[0],)) + np.log(mu/(1-mu))
def sigma(z): return 1./(1.+np.exp(-z))
Pv1 = np.zeros((Xtr.shape[0],2)); Pv1[:,0]=1-mu; Pv1[:,1]=mu;
Pv2 = np.zeros((Xva.shape[0],2)); Pv2[:,0]=1-mu; Pv2[:,1]=mu;
#Pe2 = np.zeros((Xe.shape[0],2)); Pe2[:,0]=1-mu; Pe2[:,1]=mu;
np.random.seed(0)
for l in range(nUse): # this is a lot faster than the bagging loop:
    dJ = 1.*Ytr - sigma(Ft)
    tree = ml.dtree.treeRegress(Xtr,dJ, maxDepth=) # train and save predictions
    Ft += step*tree.predict(Xtr)
    Fv += step*tree.predict(Xva)
    #Fe += step*tree.predict(Xe)
    Pv1[:,1] = sigma(Ft); Pv1[:,0] = 1-Pv1[:,1]
    Pv2[:,1] = sigma(Fv); Pv2[:,0] = 1-Pv2[:,1]
    #Pe2[:,1] = sigma(Fe); Pe2[:,0] = 1-Pe2[:,1]
    if (l & (l-1) == 0): print("Train {}; est AUC {}".format(l, dummy(Xtr,Ytr,Pv1).auc(Xtr,Ytr)))
    if (l & (l-1) == 0): print("Valid {}; est AUC {}".format(l, dummy(Xva,Yva,Pv2).auc(Xva,Yva)))
    #print " {} trees: MSE ~ {}".format(l+1, ((Yv-Pv2)**2).mean())
toKaggle('Pv1.csv',Pv1[:,1])
toKaggle('Pv2.csv',Pv2[:,1])
print("1: GradBoost, {} trees: AUC ~ {}".format(nUse, dummy(Xtr,Ytr,Pv1).auc(Xtr,Ytr)))
print("2: GradBoost, {} trees: AUC ~ {}".format(nUse, dummy(Xva,Yva,Pv2).auc(Xva,Yva)))

Train 0; est AUC 0.6917786926304814
Valid 0; est AUC 0.6995142073489291
Train 1; est AUC 0.6917786926304814
Valid 1; est AUC 0.6995142073489291
Train 2; est AUC 0.6925452741466369
Valid 2; est AUC 0.6988112773553142
Train 4; est AUC 0.6988981489833279
Valid 4; est AUC 0.7019495196493876
Train 8; est AUC 0.6990035518144547
Valid 8; est AUC 0.7024247910257154
Train 16; est AUC 0.6999923982888208
Valid 16; est AUC 0.7060600728507576
Train 32; est AUC 0.7167196233635756
Valid 32; est AUC 0.7246917992105416
Train 64; est AUC 0.7363391293544616
Valid 64; est AUC 0.74297251407674
Train 128; est AUC 0.7477803854294485
Valid 128; est AUC 0.7452654263655889
Train 256; est AUC 0.767444707478779
Valid 256; est AUC 0.7471737679224473
1: GradBoost, 274 trees: AUC ~ 0.768700748428176
2: GradBoost, 274 trees: AUC ~ 0.7464454417484182


In [20]:
Pv1 = np.genfromtxt('Pv1.csv', delimiter=',', skip_header = 1)[:, 1:2]
Pv2 = np.genfromtxt('Pv2.csv', delimiter=',', skip_header = 1)[:, 1:2]

In [47]:
print(Yva.shape, Pv1.shape, Pv2.shape, Pv3.shape, Y.shape)

(1485,) (5938, 1) (1485, 1) (7423, 1) (7423,)


In [48]:
Sv=np.hstack((Pv1,Pv2))
stack=ml.linearC.linearClassify(Sv,Y,reg=1e-3)
print("** Stacked AUC: ",stack.auc(Sv,Y))
#Se=np.hstack((Pe0,Pe1,Pe2,Pe3,Pe4))
#PeS=stack.predictSoft(Sv)
#toKaggle('Ex_Stack.csv',PeS[:,1])

** Stacked AUC:  0.7643624117804564


In [5]:
nUse= 274
mu = Y.mean()
step = 0.05
F = np.zeros((X.shape[0],)) + np.log(mu/(1-mu))
def sigma(z): return 1./(1.+np.exp(-z))
Pv = np.zeros((X.shape[0],2)); Pv[:,0]=1-mu; Pv[:,1]=mu;
np.random.seed(0)
for l in range(nUse): # this is a lot faster than the bagging loop:
    dJ = 1.*Y - sigma(F)
    tree = ml.dtree.treeRegress(X,dJ, maxDepth=10) # train and save predictions
    F += step*tree.predict(X)
    Pv[:,1] = sigma(F); Pv[:,0] = 1-Pv[:,1]
    if (l & (l-1) == 0): print("Train {}; est AUC {}".format(l, dummy(X,Y,Pv).auc(X,Y)))
    #print " {} trees: MSE ~ {}".format(l+1, ((Yv-Pv2)**2).mean())
toKaggle('Pv_m1.csv',Pv[:,1])
print("1: GradBoost, {} trees: AUC ~ {}".format(nUse, dummy(X,Y,Pv).auc(X,Y)))

Train 0; est AUC 0.8455965952817537
Train 1; est AUC 0.8504158900597607
Train 2; est AUC 0.8547874076711435
Train 4; est AUC 0.858824263126133
Train 8; est AUC 0.8671440971970129
Train 16; est AUC 0.8827218782144847
Train 32; est AUC 0.8994697969854196
Train 64; est AUC 0.9273911627409901
Train 128; est AUC 0.9642390307481157
Train 256; est AUC 0.9932481664913554
1: GradBoost, 274 trees: AUC ~ 0.9944504672419511


In [51]:
# logistic nabhan

from IPython import display

fig = plt.figure()
linc = ml.linearC.linearClassify(X,Y, initStep=0.009, reg=1e-3)
Pv0 = linc.predictSoft(X)
toKaggle('Pv_n0.csv',Pv0[:,1])
print("Linear Regress: AUC ~", linc.auc(X,Y))

C:\Users\myese\Desktop\CS 178\hw\project\CS178-Project\mltools\linearC.py:122: RuntimeWarning: invalid value encountered in true_divide
  sigx  = np.exp(respi) / (1.0+np.exp(respi))
C:\Users\myese\Desktop\CS 178\hw\project\CS178-Project\mltools\linearC.py:121: RuntimeWarning: invalid value encountered in greater
  yhati = 1.0 if respi > 0 else 0.0   # convert to 0/1 prediction


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [52]:
# bisher's code

nUse = 284
M = X.shape[0]
Pv3 = np.zeros((X.shape[0],2))
np.random.seed(0)

for l in range(nUse):
    Xi,Yi = ml.bootstrapData(X,Y, M) # draw this member's random sample of data
    tree = ml.dtree.treeClassify() # and train the model on that draw
    tree.train(Xi,Yi,minLeaf=10,maxDepth=30,nFeatures=40)
    Pv3 += tree.predictSoft(X)
    if (l & (l-1) == 0):
        print ("Train {}; est AUC {}".format(l, dummy(X,Y,Pv3/(l+1)).auc(X,Y)))
Pv3 /= nUse
toKaggle('Pv_b0.csv',Pv3[:,1])
print ("1: Random forest ({} members): Est AUC: {}".format(nUse,dummy(X,Y,Pv3).auc(X,Y)))

Train 0; est AUC 0.8326295711387415
Train 1; est AUC 0.8939292669624597
Train 2; est AUC 0.9251334002052624
Train 4; est AUC 0.9534185418008607
Train 8; est AUC 0.9754505578501236
Train 16; est AUC 0.9859953017339268
Train 32; est AUC 0.9902920022605577
Train 64; est AUC 0.9924899038682139
Train 128; est AUC 0.9936188032867534
Train 256; est AUC 0.9940941331789123
1: Random forest (284 members): Est AUC: 0.9940790318168274
